In [15]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SPEC_OBJ_DIR = '../manufactur/special_obj/'
GENE_IMG_WITH_SPECIAL_OBJ = '../generated/img_with_special_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '../generated/xml_with_special_obj/'

def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img

def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

In [32]:
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    print(xmlobj_old)
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_xml_info_under_object(attribut_vale_dic):
    element_object = ET.Element('object')
    if(len(attribut_vale_dic) == 0):
        return False
    for key in attribut_vale_dic.keys():
        tag_name = ET.SubElement(element_object, key)
        tag_name.text = attribut_vale_dic[key]
        #print(tag_name.text)
    print(element_object)
    return element_object
#generate_xml_info_under_object({'name':'my-class','difficult':'0'})

In [37]:
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)
classes = ['person', 'bicycle', 'bottle', 'cup']
for xml_name in seed_xml_names:
    img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = SEED_XML_DIR + xml_name
    #print(img_name)
    #print(xml_name)
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    for cls in special_obj_cls:
        print(cls)
        if(not cls in classes):
            continue
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue
            img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
            print(img_obj.shape)
            dic_ = {'name':cls, 'difficult':'0'}
            print(dic_)
            obj_element = generate_xml_info_under_object(dic_)
            print(obj_element)

            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)       
            tree = new_xml_etree
            img = np.array(new_pil_img)
    ## save xml and img
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S_")
    new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
    new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + time_str + img_name.split('/')[-1])
    new_xml_etree.write(GENE_XML_WITH_SPECIAL_OBJ + time_str + xml_name.split('/')[-1])
    break

person
(150, 210, 3)
{'name': 'person', 'difficult': '0'}
<Element 'object' at 0x7f1f26971638>
<Element 'object' at 0x7f1f26971638>
<Element 'object' at 0x7f1f26971638>
